In [ ]:
!pip install lifelines

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.2/349.2 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.2/94.2 kB 12.7 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4031 sha256=e4ec4383ea3438ea737fb5f1d5a76ee8c415334cfd85824012adf1ebdcf4ae37
  Stored in directory: /root/.cache/pip/wheels/25/cc/e0/ef2969164144c899fedb22b338f6703e2b9cf46eeebf254991
Successfully built autograd-gamma


In [ ]:
import pandas as pd
import numpy as np
from lifelines import CoxPHFitter

# 가상의 데이터셋 생성
np.random.seed(413)


n_customers = 1000
customer_ids = range(1, n_customers + 1)
before_7_buy_yn = np.random.randint(2, size=n_customers)
sent_mkt_msg = np.random.randint(1, 6, size=n_customers)
discount_rate = np.random.choice([0, 5, 10, 15, 20], size=n_customers)
period_days = np.random.randint(1, 21, size=n_customers)
is_buy = np.random.randint(2, size=n_customers)

data = {
    'before_7_buy_yn': before_7_buy_yn,
    'sent_mkt_msg': sent_mkt_msg,
    'discount_rate': discount_rate,
    'period_days': period_days,
    'is_buy': is_buy
}

df = pd.DataFrame(data)

# 조건 추가
df['sent_mkt_msg'] = np.where((df['sent_mkt_msg'] < 3) & (df['discount_rate'] > 10), np.random.randint(4, 6, size=n_customers), df['sent_mkt_msg'])
df['discount_rate'] = np.where((df['sent_mkt_msg'] >= 3) & (df['discount_rate'] <= 10), np.random.choice([15, 20], size=n_customers), df['discount_rate'])

# 모델 Fitting
cph = CoxPHFitter()
cph.fit(df, duration_col='period_days', event_col='is_buy')

# 회귀 계수 출력
cph.print_summary()

<lifelines.CoxPHFitter: fitted with 1000 total observations, 514 right-censored observations>
             duration col = 'period_days'
                event col = 'is_buy'
      baseline estimation = breslow
   number of observations = 1000
number of events observed = 486
   partial log-likelihood = -2900.85
         time fit was run = 2024-02-10 01:36:31 UTC

---
                  coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                               
before_7_buy_yn   0.22       1.24       0.09             0.04             0.39                 1.04                 1.48
sent_mkt_msg      0.16       1.17       0.05             0.06             0.26                 1.06                 1.29
discount_rate    -0.03       0.97       0.01            -0.05            -0.01                 0.95                 0.99

                  cmp to     z      p   -log2(p)
covariate                                       
before_7_buy_yn     0.00  2.35   0.02       5.75
sent_mkt_msg        0.00  3.12 <0.005       9.11
discount_rate       0.00 -3.02 <0.005       8.62
---
Concordance = 0.55
Partial AIC = 5807.70
log-likelihood ratio test = 15.18 on 3 df
-log2(p) of ll-ratio test = 9.23

In [ ]:
df.to_csv("survival_df.csv", index=False)  # CSV 파일로 저장합니다. 원하는 형식으로 저장하셔도 됩니다.